In [1]:
from pym import func as pym

# pym - A math and numerical methods library in python

Last Updated on 11/12/15

Written by [Alex Hagen](http://alexhagen.github.io)

Hosted at [github.com/alexhagen/pym](http://github.com/alexhagen/pym)

Documentation at [alexhagen.github.io/pym/docs](docs/)

`pym` (pronounced <i>pim</i>) is a pretty simple numerical methods library for
python.  It can be used to do interpolation, extrapolation, integration,
normalization, etc.  In general, it is a replication of Brian Bradie's book
*A Friendly Introduction to Numerical Methods* in code.  Usage should be fairly
simple, with the documentation [documentation](docs/) providing a practical
guide to using the library.

## Pym Demonstrations and Screenshots

### Installation

To install `pym`, all we have to do is install `numpy`, `scipy`, and
`matplotlib`, then download `pym` to our code directory (or wherever, really).
To do this, we can use

```bash

$ pip install numpy
$ pip install scipy
$ pip install matplotlib
$ pip install colours
$ cd ~/code
$ git clone https://github.com/alexhagen/pym.git

```

and then, we can use the library within any script by using

### Curve creation and graphing using [pyg](../pyg/)

The basis of `pym` is the `curve` class, which holds x and y data, as well as
its associated error.  We can create a function with a sinusoid in it by using
the following code

In [7]:
from pym import func as pym
import numpy as np

# use numpy to create some trigonometric functions across two periods
x_data = np.linspace(0., 4. * np.pi, 1000)
sin_data = np.sin(x_data)
cos_data = np.cos(x_data)

# define these data as ahm.curves to expose the interface to the numerical
# methods
sin = pym.curve(x_data, sin_data, name='$\sin \left( x \right)$')
cos = pym.curve(x_data, cos_data, name='$\cos \left( x \right)$')

# Plot these using the function pym.plot which returns a pyg.ah2d object
plot = sin.plot(linecolor='#285668', linestyle='-')
plot = cos.plot(linecolor='#FC8D82', linestyle='-', addto=plot)

# make it pretty with some shading, lines, changing limits, and labels
plot.fill_between(x_data, sin_data, cos_data, fc='#ccccff')
plot.lines_on()
plot.markers_off()
plot.ylim(-1.1, 1.1)
plot.xlim(0., 4. * np.pi)
plot.xlabel('$x$')
plot.ylabel('$y$')

# export it to a websvg (which doesnt convert text to paths)
plot.export('_static/curve_plotting', sizes=['2'])
plot.show('some caption')

### Integration and normalization

One of the useful options of `pym` is the ability to normalize a function,
either according to its maximum, or according to its integral.  The following is
and example of this integration, showing that after integration, we attain an
integral of 1.0.

In [6]:
# use numpy to create a monotonic function to play with
x_data = np.linspace(0., 2., 1000)
y_data = np.power(x_data, 2)

# define these data as ahm.curves to expose the interface to the numerical
# methods
y = pym.curve(x_data, y_data, name='$x^{2}$')

# Plot the unmodified function, shade the integral, and add a pointer with the
# integral value
plot = y.plot(linecolor='#285668', linestyle='-')
plot.fill_between(x_data, np.zeros_like(y_data), y_data, fc='#ccccff')
plot.add_data_pointer(1.5, point=1.5,
                      string=r'$\int f \left( x \right) dx = %.2f$' %
                      (y.integrate(0, 2)), place=(0.5, 3.))
plot.lines_on()
plot.markers_off()

# now normalize the curve with respect to the integral
y.normalize('int')
# Plot the modified function, shade the integral, and add a pointer with the
# integral value
plot = y.plot(addto=plot, linecolor='#FC8D82', linestyle='-')
plot.fill_between(x_data, np.zeros_like(y.x), y.y, fc='#ffdddd')
plot.add_data_pointer(1.25, point=0.125,
                      string=r'$\int f_{norm} \left( x \right) dx = %.2f$' %
                      (y.integrate(0, 2)), place=(0.25, 1.5))
plot.lines_on()
plot.markers_off()
plot.xlabel('$x$')
plot.ylabel('$y$')
plot.ylim(0.0, 4.0)
plot.xlim(0.0, 2.0)

# export it to a websvg (which doesnt convert text to paths)
plot.export('_static/int_norm')
plot.show('some caption')

### Curve arithmetic

Curve arithmetic (as in adding curves like `curve1 + curve2` or multiplying
them with `curve1 * curve2`) is coming as soon as I can code it (or need it and
so have to code it).

In [12]:
one = sin * sin + cos * cos
one.name = 'identity'
sin2 = sin * sin
cos2 = cos * cos

plot = one.plot(linestyle='-', linecolor='#000000')
plot = sin2.plot(linestyle='-', linecolor='#777777', addto=plot)
plot = cos2.plot(linestyle='-', linecolor='#999999', addto=plot)

plot.fill_between(sin2.x, np.zeros_like(sin2.y), sin2.y, fc='#777777')
plot.fill_between(cos2.x, cos2.y, np.ones_like(cos2.y), fc='#999999')

plot.markers_off()
plot.lines_on()

plot.export('_static/identity')
plot.show('some caption')